In [1]:
!git clone https://github.com/garydoranjr/misvm.git
!python misvm/setup.py install
!pip install numpy scikit-learn scipy tensorflow==2.12.0 mil

Cloning into 'misvm'...
remote: Enumerating objects: 198, done.
remote: Total 198 (delta 0), reused 0 (delta 0), pack-reused 198 (from 1)
Receiving objects: 100% (198/198), 180.87 KiB | 954.00 KiB/s, done.
Resolving deltas: 100% (109/109), done.
running install
/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        *****

In [2]:
# importing all the datasets modules
from mil.data.datasets import loader

# load the datasets
(corel_dogs_bags_train, corel_dogs_y_train), (corel_dogs_bags_test, corel_dogs_y_test) = loader.load_data('/usr/local/lib/python3.11/dist-packages/mil/data/datasets/csv/corel_dogs.csv')
(musk1_bags_train, musk1_y_train), (musk1_bags_test, musk1_y_test) = loader.load_data('/usr/local/lib/python3.11/dist-packages/mil/data/datasets/csv/musk1.csv')
(elephant_bags_train, elephant_y_train), (elephant_bags_test, elephant_y_test) = loader.load_data('/usr/local/lib/python3.11/dist-packages/mil/data/datasets/csv/elephant.csv')

In [3]:
# importing bag_representation
from mil.bag_representation import MILESMapping
# importing validation strategy
from mil.validators import LeaveOneOut
# importing final model, which in this case is the SVC classifier from sklearn
from mil.models import SVC
# importing trainer
from mil.trainer import Trainer
# importing preprocessing
from mil.preprocessing import StandarizerBagsList
# importing metrics, which in this case are from tf keras metrics
from mil.metrics import AUC

# instantiate trainer
trainer = Trainer()

# preparing trainer
metrics = ['acc', AUC]
model = SVC(kernel='linear', C=1, class_weight='balanced')
pipeline = [('scale', StandarizerBagsList()), ('disc_mapping', MILESMapping())]
trainer.prepare(model, preprocess_pipeline=pipeline ,metrics=metrics)

# fitting trainer
valid = LeaveOneOut()
history = trainer.fit(musk1_bags_train, musk1_y_train, sample_weights='balanced', validation_strategy=valid, verbose=1)

# printing validation results for each fold
print(history['metrics_val'])

# predicting metrics for the test set
trainer.predict_metrics(musk1_bags_test, musk1_y_test)

73/73 [==============================] - 18s 247ms/step - train_accuracy: 1.0000 - train_auc: 1.0000 - val_accuracy: 0.0000e+00 - val_auc: 0.0000e+00
[{'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 0.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 0.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 0.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 0.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 0.0, 'auc': 0.0}, {'accuracy':

{'accuracy': 0.68421054, 'auc': 0.72727275}

In [4]:
!rm -rf misvm